In [1]:
#ETL 

In [2]:
import pandas as pd

from config import API_KEY as key
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import time
import pprint

from bs4 import BeautifulSoup

In [3]:
# pull 2 data sources 

# 3. beautifulSoup 
#review webscrapper.py


In [4]:
#data = list(csv.reader(open(‘data.csv'))) # convert a csv to a 2d list


eth_path = '../data/coin_Ethereum.csv'

eth = pd.read_csv(eth_path)

#load csv
df = pd.DataFrame(eth)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2031 entries, 0 to 2030
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   SNo        2031 non-null   int64  
 1   Name       2031 non-null   object 
 2   Symbol     2031 non-null   object 
 3   Date       2031 non-null   object 
 4   High       2031 non-null   float64
 5   Low        2031 non-null   float64
 6   Open       2031 non-null   float64
 7   Close      2031 non-null   float64
 8   Volume     2031 non-null   float64
 9   Marketcap  2031 non-null   float64
dtypes: float64(6), int64(1), object(3)
memory usage: 158.8+ KB


In [5]:
df

,SNo,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap
0,1,Ethereum,ETH,2015-08-08 23:59:59,2.798810,0.714725,2.793760,0.753325,6.741880e+05,4.548689e+07
1,2,Ethereum,ETH,2015-08-09 23:59:59,0.879810,0.629191,0.706136,0.701897,5.321700e+05,4.239957e+07
2,3,Ethereum,ETH,2015-08-10 23:59:59,0.729854,0.636546,0.713989,0.708448,4.052830e+05,4.281836e+07
3,4,Ethereum,ETH,2015-08-11 23:59:59,1.131410,0.663235,0.708087,1.067860,1.463100e+06,6.456929e+07
4,5,Ethereum,ETH,2015-08-12 23:59:59,1.289940,0.883608,1.058750,1.217440,2.150620e+06,7.364501e+07
...,...,...,...,...,...,...,...,...,...,...
2026,2027,Ethereum,ETH,2021-02-23 23:59:59,1781.409018,1378.840907,1781.409018,1570.203925,5.202986e+10,1.802510e+11
2027,2028,Ethereum,ETH,2021-02-24 23:59:59,1710.983795,1511.018963,1571.476497,1626.575705,3.132900e+10,1.867440e+11
2028,2029,Ethereum,ETH,2021-02-25 23:59:59,1670.224062,1465.058946,1625.393882,1475.703735,2.448168e+10,1.694427e+11
2029,2030,Ethereum,ETH,2021-02-26 23:59:59,1559.028891,1407.979253,1478.653270,1446.033650,3.143600e+10,1.660555e+11


## api

In [6]:
#api

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'100',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': key,
}



session = Session()
session.headers.update(headers)

data = {}

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
#   print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

#remove status dict 
cleaned_data = data['data']
# print(cleaned_data)

# def subset(headers):
#     #list of return rows 
#     results = []
    
#     for row in cleaned_data:
#         query = []
#         # loop through keys for each row, find interested headers
#         # keys is a built in method for dict
#         for key in row.keys():
#             if key in headers:
#                 #cast value to str 
#                 query.append(str(row[key]))
#         #format results by adding ',' and convert to str
#         formatted_row = ','.join(query)
        
#         results.append(formatted_row)
#     return results
        

def subset(headers):
    #list of return rows 
    results = []
    
    #check if header values is nested or not 
    nested_header = []
    
    #remember headers that cant be found in main query, then must be nested 
    for header in headers:
        #use .keys method on dict, returns all headers 
        if header not in cleaned_data[0].keys():
            nested_header.append(header)
            
    
    for row in cleaned_data:
        query = []
        # loop through keys for each row, find interested headers
        # keys is a built in method for dict
        for key in row.keys():
            if key in headers:
                #cast value to str 
                query.append(str(row[key]))
#             else:
#                 for nested_header in row['quote']['USD'].keys():
#                     if nested_header in headers:
#                         query.append(str(row['quote']['USD'][nested_header]))
        
        for header in nested_header:
            if header in row['quote']['USD'].keys():
                query.append(str(int(row['quote']['USD'][header])))       
        #format results by adding ',' and convert to str
        formatted_row = ','.join(query)
        results.append(formatted_row)
    return results
        

                
# data_eth = []

# for row in cleaned_data:
# #     print(row['id'] + ' : ' + row['slug'])
# #     print(f"{row['id']} : {row['slug']}")
    
#     if 'ethereum' in row['slug']:
#         data_eth.append(f"{row['id']} : {row['slug']}")
        
# for row in data_eth:
#     print(row)


get_data = subset(['symbol','price'])

for row in get_data:
    print(row)




BTC,36450
ETH,2574
USDT,1
BNB,368
ADA,1
DOGE,0
XRP,0
USDC,1
DOT,22
UNI,24
BCH,624
SOL,42
LTC,170
LINK,25
ICP,80
BUSD,0
MATIC,1
THETA,9
XLM,0
VET,0
ETC,60
WBTC,36459
FIL,76
TRX,0
EOS,5
DAI,1
XMR,268
AAVE,330
KSM,470
NEO,49
MIOTA,1
MKR,3264
CAKE,18
BSV,169
ALGO,1
FTT,33
TFUEL,0
CRO,0
XTZ,3
ATOM,13
SHIB,0
KLAY,1
BTCB,36267
AVAX,15
HT,14
BTT,0
LUNA,5
LEO,2
RUNE,9
UST,0
COMP,358
HBAR,0
DCR,137
DASH,173
CEL,7
ZEC,142
TEL,0
EGLD,91
XEM,0
WAVES,14
YFI,39462
HOT,0
TUSD,1
CHZ,0
NEAR,3
SUSHI,10
ZIL,0
SNX,10
NEXO,2
HNT,13
BTG,66
PAX,1
MANA,0
ENJ,1
STX,0
BAT,0
ZEN,87
MDX,2
QTUM,9
REV,0
NANO,7
DGB,0
ONE,0
CRV,2
GRT,0
OKB,14
BNT,4
ONT,0
HUSD,1
ZRX,0
BAKE,3
SC,0
UMA,12
OMG,5
FTM,0
CELO,2
BCD,3
CHSB,0
RVN,0
KCS,8


In [7]:

# url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest'
# parameters = {
#   'slug':'bitcoin',
#   'limit':'100',
#   'convert':'USD'
# }
# headers = {
#   'Accepts': 'application/json',
#   'X-CMC_PRO_API_KEY': '1c07eee4-35f6-405c-8c43-aeb0b8682976',
# }


# response = session.get(url, params=parameters)
# print((response))
# pprint.pprint(json.loads(response.text))


In [8]:
# transform
# perform data operation 
# dedupe, remove outliers 

In [9]:
# Load 

#SQL, postgres 